In [1]:
import numpy as np


#### Complex numbers:
2 May 2021

In [2]:
1j * 1j

(-1+0j)

In [3]:
b = complex(1,2)

In [4]:
1j * b

(-2+1j)

In [5]:
b.real

1.0

In [6]:
b.imag

2.0

In [7]:
b.conjugate()

(1-2j)

In [8]:
abs(b)

2.23606797749979

In [9]:
pow(b,2)

(-3+4j)

In [10]:
import cmath
cmath.sin(b)

(3.165778513216168+1.959601041421606j)

#### GBM example:

**Questions:** 
1. Why is the approximation with a coarser time grid more precise? 
2. A more efficient way to set the random seed instead of repeating the instruction to do so? 
3. How can I use validation in a longer example? Shall I validate also f_diff and S_diff at each time step of the forward mode? How to validate f_bar and S_bar at each time step of the reverse mode?

In [11]:
# parameters
S = 1
sigma = 0.2
r = 0.05
h = 0.1 # time-step 
T = 1 # maturity

In [12]:
def GBM(S, sigma, r, T, t=0):
    # simulates one value of the GBM at maturity
    Z = np.random.normal()
    return S * np.exp((r - sigma/2)*(T-t) + sigma * Z * (T-t))

In [13]:
# Euler-Maruyama approximation
def GBM_Euler_Maruyama(S, sigma, r, T, h, t=0):
    # simulates one path of GBM
    assert(T/h % 1 == 0)
    N = int(T/h)
    for n in range(N):
        Z = np.random.normal()
        f = 1 + r*h + sigma * np.sqrt(h) * Z
        S = f * S
    return S 

In [14]:
np.random.seed(1)
GBM_Euler_Maruyama(S, sigma, r, T, 0.000001)

1.173958356701858

In [188]:
np.random.seed(1)
GBM_Euler_Maruyama(S, sigma, r, T, 0.0001)

1.252949258349744

In [189]:
np.random.seed(1)
GBM_Euler_Maruyama(S, sigma, r, T, 0.001)

1.3180921666317842

In [190]:
np.random.seed(1)
GBM(S,sigma, r,T)

1.3163583163874353

##### Validation of sensitivities:

###### Bumping

To do: plot bumping error wrt to epsilon (find sweet spot at e^3 = 10^(-16))

In [15]:
epsilon = 10e-6

#np.random.seed(1)
#S_minus = GBM(S-epsilon, sigma, r, T)
#np.random.seed(1)
#S_plus = GBM(S+epsilon, sigma, r, T)
#delta_bumping = (S_plus-S_minus) / (2*epsilon)

#np.random.seed(1)
#sigma_plus = GBM(S, sigma+epsilon, r, T)
#np.random.seed(1)
#sigma_minus = GBM(S, sigma-epsilon, r, T)
#vega_bumping = (sigma_plus-sigma_minus) / (2*epsilon)

#print("Approximation of delta in GBM is: ", delta_bumping)
#print("Approximation of vega in GBM is: ", vega_bumping)

np.random.seed(1)
S_minus = GBM_Euler_Maruyama(S-epsilon, sigma, r, T, h)
np.random.seed(1)
S_plus = GBM_Euler_Maruyama(S+epsilon, sigma, r, T, h)
delta_bumping = (S_plus-S_minus) / (2*epsilon)

np.random.seed(1)
sigma_plus = GBM_Euler_Maruyama(S, sigma+epsilon, r, T, h)
np.random.seed(1)
sigma_minus = GBM_Euler_Maruyama(S, sigma-epsilon, r, T, h)
vega_bumping = (sigma_plus-sigma_minus) / (2*epsilon)

print("Approximation of delta in GBM with Euler-Maruyama using bumping is: ", delta_bumping)
print("Approximation of vega in GBM with Euler-Maruyama using bumping is: ", vega_bumping)

Approximation of delta in GBM with Euler-Maruyama using bumping is:  0.9607412490286736
Approximation of vega in GBM with Euler-Maruyama using bumping is:  -0.573796331970744


###### Complex variable trick

In [16]:
np.random.seed(1)
delta_complex_trick = (GBM_Euler_Maruyama(S + epsilon * 1j, sigma, r, T, h)).imag / epsilon
np.random.seed(1)
vega_complex_trick = (GBM_Euler_Maruyama(S, sigma + epsilon * 1j, r, T, h)).imag / epsilon
print("Approximation of delta in GBM with Euler-Maruyama using the complex variable trick is:", delta_complex_trick)
print("Approximation of vega in GBM with Euler-Maruyama using the complex variable trick is:", vega_complex_trick)

Approximation of delta in GBM with Euler-Maruyama using the complex variable trick is: 0.960741249027779
Approximation of vega in GBM with Euler-Maruyama using the complex variable trick is: -0.5737963320088069


##### Forward Mode:

In [17]:
def forward_mode_GBM_Euler_Maruyama(S_diff, sigma_diff, S, sigma, r, T, h, t=0):
    assert((S_diff==0 or sigma_diff==0) and (S_diff==1 or sigma_diff==1))
    assert(T/h % 1 == 0)
    N = int(T/h)
    for n in range(N):
        Z = np.random.normal()
        f_diff = sigma_diff * np.sqrt(h) * Z
        f = 1 + r*h + sigma * np.sqrt(h) * Z
        S_diff = f_diff * S + f * S_diff
        S = f * S
    return S_diff

In [18]:
np.random.seed(1)
delta = forward_mode_GBM_Euler_Maruyama(1, 0, S, sigma, r, T, h)
np.random.seed(1)
vega = forward_mode_GBM_Euler_Maruyama(0, 1, S, sigma, r, T, h)
print("Delta forward mode: ", delta)
print("Difference to bumping: ", np.abs(delta-delta_bumping))
print("Difference to complex variable trick: ", np.abs(delta-delta_complex_trick))
print("-------------")
print("Vega forward mode: ", vega)
print("Difference to bumping: ", np.abs(vega-vega_bumping))
print("Difference to complex variable trick", np.abs(vega-vega_complex_trick))
print("-------------")
print("Nice improvement in accuracy using complex variable trick!")

Delta forward mode:  0.9607412490277795
Difference to bumping:  8.941736240331011e-13
Difference to complex variable trick:  4.440892098500626e-16
-------------
Vega forward mode:  -0.5737963319827681
Difference to bumping:  1.2024159445900295e-11
Difference to complex variable trick 2.603872673034857e-11
-------------
Nice improvement in accuracy using complex variable trick!


##### Reverse Mode:

In [19]:
# forward step 
def forward_step_GBM_Euler_Maruyama(S, sigma, r, T, h, t=0):
    S_list = [S]
    Z_list = []
    f_list = []
    assert(T/h % 1 == 0)
    N = int(T/h)
    for n in range(N):
        Z = np.random.normal()
        f = 1 + r*h + sigma * np.sqrt(h) * Z
        S = f * S
        S_list.append(S), f_list.append(f), Z_list.append(Z)
    return S_list, f_list, Z_list

In [20]:
np.random.seed(1)
S_list, f_list, Z_list = forward_step_GBM_Euler_Maruyama(S, sigma, r, T, h)

In [21]:
# backward step
S_bar = 1
sigma_bar = 0
assert(T/h % 1 == 0)
N = int(T/h)
for n in reversed(range(N)):
    f_bar = S_list[n] * S_bar # by construction, we are using S_{n-1} on the right hand side
    S_bar = f_list[n] * S_bar 
    sigma_bar += np.sqrt(h) * Z_list[n] * f_bar

In [22]:
print("Delta reverse mode: ", S_bar, ". Difference to forward mode: ", np.abs(S_bar-delta))
print("Vega reverse mode: ", sigma_bar, ". Difference to forward mode: ", np.abs(sigma_bar-vega))

Delta reverse mode:  0.9607412490277795 . Difference to forward mode:  0.0
Vega reverse mode:  -0.5737963319827684 . Difference to forward mode:  2.220446049250313e-16


#### Least-Squares fit example: 
Goal: compute the sensitivity of the sum of squared residuals to changes the parameters of a cubic function. 

In [357]:
#parameters
a = 0.5
b = 0
c = - 1
d = 1
data = [(0,0.5), (1,0.5), (-1, 0.5), (-2,-1), (2,3), (-0.5,1.5), (0.5,0.5), (1.5, 1.5), (-1.5, 1), (0,1)]

In [358]:
def SSR(a, b, c, d, data):
    SSR = 0
    N = len(data)
    for n in range(N):
        x, y = data[n]
        f = a*x**3 + b*x**2 + c*x + d
        SSR += (f - y)**2 
    return SSR

In [359]:
SSR(a, b, c, d, data)

1.390625

##### Validation of sensitivities

###### Complex variable trick

In [360]:
epsilon = 10e-20
a_complex_trick = (SSR(a + epsilon *1j, b, c, d, data)).imag / epsilon
b_complex_trick = (SSR(a , b + epsilon *1j, c, d, data)).imag / epsilon
c_complex_trick = (SSR(a , b, c + epsilon *1j, d, data)).imag / epsilon
d_complex_trick = (SSR(a , b, c, d + epsilon *1j, data)).imag / epsilon
print("Approximation of sensitivity wrt a is:", a_complex_trick)
print("Approximation of sensitivity wrt b is:", b_complex_trick)
print("Approximation of sensitivity wrt c is:", c_complex_trick)
print("Approximation of sensitivity wrt d is:", d_complex_trick)

Approximation of sensitivity wrt a is: -2.8125000000000004
Approximation of sensitivity wrt b is: -0.2500000000000001
Approximation of sensitivity wrt c is: -2.25
Approximation of sensitivity wrt d is: 2.0


###### Exact computation

In [405]:
a_diff_exact = sum([2 * (f(x)-y) * x**3 for x,y in data])
b_diff_exact = sum([2 * (f(x)-y) * x**2 for x,y in data])
c_diff_exact = sum([2 * (f(x)-y) * x for x,y in data])
d_diff_exact = sum([2 * (f(x)-y) for x,y in data])
print("Exact value of sensitivity wrt a is: ", a_diff_exact)
print("Exact value of sensitivity wrt b is: ", b_diff_exact)
print("Exact value of sensitivity wrt c is: ", c_diff_exact)
print("Exact value of sensitivity wrt d is: ", d_diff_exact)

Exact value of sensitivity wrt a is:  -2.8125
Exact value of sensitivity wrt b is:  -0.25
Exact value of sensitivity wrt c is:  -2.25
Exact value of sensitivity wrt d is:  2.0


##### Forward mode

In [365]:
def forward_mode_least_squares(a_diff, b_diff, c_diff, d_diff, a, b, c, d, data):
    # to do: improve assertion to cover the case in which exactly one value of the var_diff is 1 and the rest is 0
    assert a_diff + b_diff + c_diff + d_diff == 1
    assert (a_diff == 1 or b_diff==1 or c_diff==1 or d_diff==1)
    SSR_diff = 0
    SSR = 0
    N = len(data)
    for n in range(N):
        x, y = data[n]
        f_diff = a_diff * x**3 + b_diff * x**2 + c_diff * x + d_diff 
        f = a*x**3 + b*x**2 + c*x + d
        SSR_diff += 2*(f-y)*f_diff 
        SSR += (f-y)**2
    return SSR_diff

In [368]:
a_diff = forward_mode_least_squares(1, 0, 0, 0, a, b, c, d, data)
b_diff = forward_mode_least_squares(0, 1, 0, 0, a, b, c, d, data)
c_diff = forward_mode_least_squares(0, 0, 1, 0, a, b, c, d, data)
d_diff = forward_mode_least_squares(0, 0, 0, 1, a, b, c, d, data)

print("Sensitivity wrt a forward mode: ", a_diff)
print("Difference to complex variable trick: ", np.abs(a_diff-a_complex_trick))
print("-------------")
print("Sensitivity wrt b forward mode: ", b_diff)
print("Difference to complex variable trick: ", np.abs(b_diff-b_complex_trick))
print("-------------")
print("Sensitivity wrt c forward mode: ", c_diff)
print("Difference to complex variable trick: ", np.abs(c_diff-c_complex_trick))
print("-------------")
print("Sensitivity wrt d forward mode: ", d_diff)
print("Difference to complex variable trick: ", np.abs(d_diff-d_complex_trick))


Sensitivity wrt a forward mode:  -2.8125
Difference to complex variable trick:  4.440892098500626e-16
-------------
Sensitivity wrt b forward mode:  -0.25
Difference to complex variable trick:  1.1102230246251565e-16
-------------
Sensitivity wrt c forward mode:  -2.25
Difference to complex variable trick:  0.0
-------------
Sensitivity wrt d forward mode:  2.0
Difference to complex variable trick:  0.0


##### Reverse mode

In [373]:
# forward step 
def SSR(a, b, c, d, data):
    SSR = 0
    x_list = [] 
    y_list = []
    SSR_list = [SSR]
    f_list = [] 
    N = len(data)
    for n in range(N):
        x, y = data[n]
        f = a*x**3 + b*x**2 + c*x + d
        SSR += (f - y)**2 
        x_list.append(x), y_list.append(y), SSR_list.append(SSR), f_list.append(f) 
    return x_list, y_list, SSR_list, f_list

In [374]:
x_list, y_list, SSR_list, f_list = SSR(a, b, c, d, data)

In [378]:
# backward step 
SSR_bar = 1
a_bar = 0
b_bar = 0
c_bar = 0
d_bar = 0 
f_bar = 0
N = len(data)
for n in reversed(range(N)):
    f_bar = 2*(f_list[n] - y_list[n])*SSR_bar 
    a_bar += x_list[n]**3 * f_bar
    b_bar += x_list[n]**2 * f_bar
    c_bar += x_list[n] * f_bar 
    d_bar += f_bar

In [384]:
print("Sensitivity wrt a reverse mode: ", a_bar, ". Difference to forward mode: ", np.abs(a_bar-a_diff))
print("Sensitivity wrt b reverse mode: ", b_bar, ". Difference to forward mode: ", np.abs(b_bar-b_diff))
print("Sensitivity wrt c reverse mode: ", c_bar, ". Difference to forward mode: ", np.abs(c_bar-c_diff))
print("Sensitivity wrt d reverse mode: ", d_bar, ". Difference to forward mode: ", np.abs(d_bar-d_diff))
print("---------------")
print("Sensitivities are computed exactly.")

Sensitivity wrt a reverse mode:  -2.8125 . Difference to forward mode:  0.0
Sensitivity wrt b reverse mode:  -0.25 . Difference to forward mode:  0.0
Sensitivity wrt c reverse mode:  -2.25 . Difference to forward mode:  0.0
Sensitivity wrt d reverse mode:  2.0 . Difference to forward mode:  0.0
---------------
Sensitivities are computed exactly.


#### To do: Ornstein–Uhlenbeck example: